## Question 3

In [1]:
# 1)
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
spark = SparkSession \
    .builder \
    .appName("ReadFromData1") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

df = lines

split_col = pyspark.sql.functions.split(df['value'], ',')
df = df.withColumn('PassengerId', split_col.getItem(0).cast(IntegerType()))
df = df.withColumn('Survived', split_col.getItem(1).cast(IntegerType()))
df = df.withColumn('Pclass', split_col.getItem(2))
df = df.withColumn('Name', split_col.getItem(3))
df = df.withColumn('Sex', split_col.getItem(4))
df = df.withColumn('Age', split_col.getItem(5).cast(DoubleType()))
df = df.withColumn('Fare', split_col.getItem(6).cast(DoubleType()))
df = df.drop('value')
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)



In [2]:
# 2)
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
query = df \
    .writeStream \
    .queryName("Query") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [3]:
query.stop()

In [4]:
part_2 = sqlContext.sql("SELECT * FROM Query")
part_2.show()

+-----------+--------+------+--------------------+------+----+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|   Fare|
+-----------+--------+------+--------------------+------+----+-------+
|          1|       0|     3| Mr.OwenHarrisBraund|  male|22.0|   7.25|
|          2|       1|     1|Mrs.JohnBradley(F...|female|38.0|71.2833|
|          3|       1|     3| Miss.LainaHeikkinen|female|26.0|  7.925|
|          4|       1|     1|Mrs.JacquesHeath(...|female|35.0|   53.1|
|          5|       0|     3|Mr.WilliamHenryAllen|  male|35.0|   8.05|
|          6|       0|     3|       Mr.JamesMoran|  male|null| 8.4583|
|          7|       0|     1| Mr.TimothyJMcCarthy|  male|54.0|51.8625|
|          8|       0|     3|Master.GostaLeona...|  male| 2.0| 21.075|
|          9|       1|     3|Mrs.OscarW(Elisab...|female|27.0|11.1333|
|         10|       1|     2|Mrs.Nicholas(Adel...|female|14.0|30.0708|
|         11|       1|     3|Miss.MargueriteRu...|female| 4.0|   16.7|
|     

In [5]:
# 3)
part_3 = sqlContext.sql("SELECT SUM(Survived)/Count(Survived)*100 As Percentage FROM Query")
part_3.show()

+-----------------+
|       Percentage|
+-----------------+
|35.15981735159817|
+-----------------+



In [6]:
# 4)
part_4 = sqlContext.sql("SELECT Sex, COUNT(Survived) AS Survived FROM Query WHERE Survived == 1 GROUP BY SEX")
part_4.show()
# Pattern: More females are survived than males

+------+--------+
|   Sex|Survived|
+------+--------+
|female|      56|
|  male|      21|
+------+--------+



In [7]:
# 5)
part_5 = sqlContext.sql("SELECT Pclass, SUM(Survived)/Count(Survived)*100 As Percentage FROM Query GROUP BY Pclass ORDER BY Pclass")
part_5.show()

+------+------------------+
|Pclass|        Percentage|
+------+------------------+
|     1| 45.23809523809524|
|     2| 45.23809523809524|
|     3|28.888888888888886|
+------+------------------+

